# Intercommodity spreads

In addition to listed calendar spreads, there are other listed spreads.
Many use the name ICS (intercommodity spread).
There are also multimonth baskets like packs and bundles, which
are not calendar spreads but are not ICS either.


## Top-volume intercommodity spreads

Let's find the most traded spreads that are not calendars among `CL` products on a specific date.
First let's find all instrument definitions.

In [1]:
from itertools import cycle

import databento as db
import plotly.express as px

from finm37000 import (
    get_databento_api_key,
    temp_env,
)

px.defaults.color_discrete_sequence = px.colors.qualitative.Set3
color_palette = cycle(px.defaults.color_discrete_sequence)

with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()

In [2]:
date = "2025-10-23"
cme = "GLBX.MDP3"

In [3]:
product = "CL"
def_df = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=date,
).to_df()

It is simple to extract the spreads from the other futures:

In [4]:
spread_df = def_df[def_df["instrument_class"] == db.InstrumentClass.FUTURE_SPREAD]

But many of these spreads are calendars, so let's filter those out.

In [5]:
calendar_ish_sub_types = [
    "SP",  # calendar spreads
    "BF",  # calendar butterfly
    "CF",  # calendar condor
]
non_calendar_spreads = spread_df[~spread_df["secsubtype"].isin(calendar_ish_sub_types)]
non_calendar_spreads[["symbol", "secsubtype"]]

,symbol,secsubtype
ts_recv,,
2025-10-23 00:00:00+00:00,CL:C1 HO-CL H7,C1
2025-10-23 00:00:00+00:00,CLN9-BZQ9,IS
2025-10-23 00:00:00+00:00,CLX5-BZF6,IS
2025-10-23 00:00:00+00:00,CLH6-OQDH6,IS
2025-10-23 00:00:00+00:00,CLX5-WSX5,IS
...,...,...
2025-10-23 00:00:00+00:00,CLX3-BZX3,IS
2025-10-23 00:00:00+00:00,CL:BZ F6-Z6,IP
2025-10-23 00:00:00+00:00,CLU6-BZV6,IS


In [6]:
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=non_calendar_spreads["symbol"],
    start=date,
).to_df()

In [7]:
ohlcv.sort_values("volume", ascending=False)[["symbol", "volume"]]

,symbol,volume
ts_event,,
2025-10-23 00:00:00+00:00,CLZ5-BZZ5,28399
2025-10-23 00:00:00+00:00,CLZ5-BZF6,12879
2025-10-23 00:00:00+00:00,CLF6-BZF6,12133
2025-10-23 00:00:00+00:00,CL:BZ F6-G6,11379
2025-10-23 00:00:00+00:00,CLM6-BZM6,9326
...,...,...
2025-10-23 00:00:00+00:00,CL:BZ G6-Z6,5
2025-10-23 00:00:00+00:00,CL:BZ V6-X6,4
2025-10-23 00:00:00+00:00,CLV6-BZV6,3


Various Treasury spreads like the `BOB`, `FYT`, etc.

In [8]:
treasury_ics = [
    "TUF",
    "TUT",
    "TUX",
    "TUB",
    "TUL",
    "FYT",
    "FIX",
    "FOB",
    "FOL",
    "TEX",
    "NOB",
    "NOL",
    "NCB",
    "NUB",
    "BOB",
]
def_df = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=[f"{product}.FUT" for product in treasury_ics],
    stype_in="parent",
    start=date,
).to_df()

spread_df = def_df[def_df["instrument_class"] == db.InstrumentClass.FUTURE_SPREAD]
non_calendar_spreads = spread_df[~spread_df["secsubtype"].isin(calendar_ish_sub_types)]
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=non_calendar_spreads["symbol"],
    start=date,
).to_df()
ohlcv.sort_values("volume", ascending=False)[["symbol", "volume"]]

,symbol,volume
ts_event,,
2025-10-23 00:00:00+00:00,FYT 03-02 Z5,6780
2025-10-23 00:00:00+00:00,NOB 02-01 Z5,5106
2025-10-23 00:00:00+00:00,TUF 06-05 Z5,2645
2025-10-23 00:00:00+00:00,NCB 03-02 Z5,2591
2025-10-23 00:00:00+00:00,TEX 07-05 Z5,1577
2025-10-23 00:00:00+00:00,NUB 02-01 Z5,1463
2025-10-23 00:00:00+00:00,BOB 02-01 Z5,1461
2025-10-23 00:00:00+00:00,TUT 02-01 Z5,1339
2025-10-23 00:00:00+00:00,FIX 02-01 Z5,569


`SOM` Soybean crush

In [9]:
product = "SOM"
def_df = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=date,
).to_df()

spread_df = def_df[def_df["instrument_class"] == db.InstrumentClass.FUTURE_SPREAD]
non_calendar_spreads = spread_df[~spread_df["secsubtype"].isin(calendar_ish_sub_types)]
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=non_calendar_spreads["symbol"],
    start=date,
).to_df()
ohlcv.sort_values("volume", ascending=False)[["symbol", "volume"]]

,symbol,volume
ts_event,,
2025-10-23 00:00:00+00:00,SOM:SI Z5-Z5-X5,249
2025-10-23 00:00:00+00:00,SOM:SI Z5-Z5-F6,76
2025-10-23 00:00:00+00:00,SOM:SI H6-H6-H6,68
2025-10-23 00:00:00+00:00,SOM:SI F6-F6-F6,41
2025-10-23 00:00:00+00:00,SOM:SI K6-K6-K6,28
2025-10-23 00:00:00+00:00,SOM:SI N6-N6-N6,17
2025-10-23 00:00:00+00:00,SOM:SI U6-U6-U6,11
2025-10-23 00:00:00+00:00,SOM:SI Z6-Z6-X6,5
2025-10-23 00:00:00+00:00,SOM:SI Q6-Q6-Q6,3


## Structure of intercommodity spreads

The listed non-calendar spreads represent fixed-size spreads or baskets
with two or more legs. Many of these are intercommodity spreads (ICS).
Typically the fixed size is determined by physical or financial relationships.

Examples:
1. Crack spreads: the spread between crude oil and its refined products, heating oil (HO aka ULSD) and /or gasoil RBOB.
    * Conversion of 42 gallons refined per barrel
2. Soybean crush spreads: the spread between soybeans and their refined products, oil and meal.
    *  Listed conversion is 10 beans to 11 meal to 9 oil.
3. Treasury ICS: spreads across maturity, for example
    * FYT: Five-year vs. ten-year (3:2 ratio)
    * BOB: T-bond vs. ultra bond (2:1 ratio)

Many more explained on the exchange web sites. Although most of the focus has been on
CME products, the ICE reference is a nice summary, whereas the CME is very detailed.

https://cmegroupclientsite.atlassian.net/wiki/spaces/EPICSANDBOX/pages/457089763/Spreads+and+Combinations+Available+on+CME+Globex#CB-Crack-Box

https://www.ice.com/publicdocs/technology/ICE_Strategy_Code_Reference_Manual.pdf

### Value of intercommodity spreads

* Reduced risk compared to single legs
* Reduced margin (at least for listed spreads)
* Focus on relative changes in supply/demand of components

### Challenges of intercommodity spreads

* Expiration cycles may not match
* Fixed size is a trading convenience but may not reflect more accurate relationship.